# Kitaev Model: Toric Code

Kitaev Hamiltonian:
$
\begin{equation}
H_K = - \sum_p B_p - \sum_s A_s
\end{equation}
$

With operators defined as:

$
\begin{align}
B_p &= \Pi_{(ij) \in p} \sigma_{(ij)}^z \\
A_s &= \Pi_{(ij) \in s} \sigma_{(ij)}^x \\
\end{align}
$

where $p$ is a plaquette of the lattice and $s$ is a star vertex of the lattice.

In [3]:
import numpy as np

Consider an $N=2$ torus, meaning there are $2N^2 = 8$ spin sites and periodic boundary conditions

In [30]:
Nspins = 8  # number of spins on lattice
Np = 4  # number of plaquettes
Ns = 4  # number of star vertices

Generate $B_p$ and $A_s$ operators

In [21]:
x = np.array([[0,1],[1,0]])
z = np.array([[1,0],[0,-1]])
iden = np.identity(2)

In [29]:
2**8

256

In [27]:
np.kron(np.kron(np.kron(x,x),x),x)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [34]:
Bp_s = []  # list of Bp operators indexed from 0 to 3
Bp_s.append(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(iden, z), z), z), iden), z), iden), iden))
Bp_s.append(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(z, iden), z), z), z), iden), iden), iden))
Bp_s.append(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(iden, z), iden), iden), iden), z), z), z))
Bp_s.append(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(z, iden), iden), iden), z), iden), z), z))

As_s = []  # list of Bp operators indexed from 0 to 3
As_s.append(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(iden, iden), x), iden), x), x), x), iden))
As_s.append(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(iden, iden), iden), x), x), x), iden), x))
As_s.append(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(x, x), x), iden), iden), iden), x), iden))
As_s.append(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(x, x), iden), x), iden), iden), iden), x))

Check $B_p$ and $A_s$ operators commute according to:

$
\begin{align}
[B_p, B_p^{\prime}] &= 0 \\
[A_s, A_s^{\prime}] &= 0 \\
[B_p, A_s] &= 0 \\
\end{align}
$

Check that periodic boundary condition:

$
\begin{align}
\Pi_s A_s &= 1 \\
\Pi_p B_p &= 1 \\
\end{align}
$


In [58]:
# Check commutation relations. Print if there something does NOT commute

# Commutation between different Bp operators
for i in range(len(Bp_s) - 1):
    if not np.array_equal(np.matmul(Bp_s[i], Bp_s[i+1]), np.matmul(Bp_s[i+1], Bp_s[i])):
        print('Bp', i, 'does not commute with Bp', i+1)

# Commutation between different As operators
for i in range(len(As_s) - 1):
    if not np.array_equal(np.matmul(As_s[i], As_s[i+1]), np.matmul(As_s[i+1], As_s[i])):
        print('As', i, 'does not commute with As', i+1)

# Commutation between different Bp and As operators
for i in range(len(As_s)):
    for j in range(len(Bp_s)):
        if not np.array_equal(np.matmul(As_s[i], Bp_s[j]), np.matmul(Bp_s[j], As_s[i])):
            print('As', i, 'does not commute with As', i+1)

# Check periodic boundary condition
Bp_product = np.identity(Bp_s[0].shape[0])
for i in range(len(Bp_s)):
    Bp_product = np.matmul(Bp_product, Bp_s[i])
print('Is the product of Bp operators equal to the identity matrix?:', np.array_equal(Bp_product, np.identity(Bp_s[0].shape[0])))

As_product = np.identity(As_s[0].shape[0])
for i in range(len(As_s)):
    As_product = np.matmul(As_product, As_s[i])
print('Is the product of As operators equal to the identity matrix?:', np.array_equal(As_product, np.identity(As_s[0].shape[0])))

Is the product of Bp operators equal to the identity matrix?: True
Is the product of As operators equal to the identity matrix?: True


Looks like everything checks out. Now to compute the Hamiltonian $H_K$